## Using SQL to Get Insights on Sales Data

![Parked motorcycle](motorcycle.jpg)

This company sells motorcycle parts, and they've asked for some help in analyzing their sales data!

They operate three warehouses in the area, selling both retail and wholesale. They offer a variety of parts and accept credit cards, cash, and bank transfer as payment methods. However, each payment type incurs a different fee.

The board of directors wants to **gain a better understanding of wholesale revenue by product line, and how this varies month-to-month and across warehouses**. The task is to calculate net revenue for each product line and grouping results by month and warehouse. The results should be filtered so that only `"Wholesale"` orders are included.

They have provided access to their database, which contains the following table called `sales`:

### Sales
| Column | Data type | Description |
|--------|-----------|-------------|
| `order_number` | `VARCHAR` | Unique order number. |
| `date` | `DATE` | Date of the order, from June to August 2021. |
| `warehouse` | `VARCHAR` | The warehouse that the order was made from&mdash; `North`, `Central`, or `West`. |
| `client_type` | `VARCHAR` | Whether the order was `Retail` or `Wholesale`. |
| `product_line` | `VARCHAR` | Type of product ordered. |
| `quantity` | `INT` | Number of products ordered. | 
| `unit_price` | `FLOAT` | Price per product (dollars). |
| `total` | `FLOAT` | Total price of the order (dollars). |
| `payment` | `VARCHAR` | Payment method&mdash;`Credit card`, `Transfer`, or `Cash`. |
| `payment_fee` | `FLOAT` | Percentage of `total` charged as a result of the `payment` method. |


The query output should be presented in the following format:

| `product_line` | `month` | `warehouse` |	`net_revenue` |
|----------------|-----------|----------------------------|--------------|
| product_one | --- | --- | --- |
| product_one | --- | --- | --- |
| product_one | --- | --- | --- |
| product_one | --- | --- | --- |
| product_one | --- | --- | --- |
| product_one | --- | --- | --- |
| product_two | --- | --- | --- |
| ... | ... | ... | ... |

### Top Performing Product Lines

In [7]:
SELECT product_line, 
       (CASE 
            WHEN EXTRACT('month' FROM date) = 6 THEN 'June'
            WHEN EXTRACT('month' FROM date) = 7 THEN 'July'
            WHEN EXTRACT('month' FROM date) = 8 THEN 'August'
        END) AS month,
       warehouse, 
       ROUND(SUM(total * (1 - payment_fee))::numeric, 2)  AS net_revenue
FROM sales
WHERE client_type = 'Wholesale'
GROUP BY product_line, month, warehouse
ORDER BY product_line ASC, month ASC, net_revenue DESC;

,product_line,month,warehouse,net_revenue
0,Breaking system,August,Central,3009.10
1,Breaking system,August,West,2475.71
2,Breaking system,August,North,1753.19
3,Breaking system,July,Central,3740.94
4,Breaking system,July,West,3030.39
5,Breaking system,July,North,2568.55
6,Breaking system,June,Central,3648.14
7,Breaking system,June,North,1472.93
8,Breaking system,June,West,1200.64
9,Electrical system,August,North,4673.99


Some insights from the table above:

1. **Top-Performing Product Lines**: By examining which product lines generate the highest net revenue, you can identify your top-performing product categories. For example, "Engine" products consistently generate high net revenue, especially in the "Central" and "North" warehouses.
2. **Seasonal Trends**: By examining net revenue across different months, it's possible to identify seasonal trends. For example, "Suspension & traction" products in the "Central" warehouse show significantly higher net revenue in June compared to August. 
3. **Warehouse Diversification**: Understanding which product lines perform well in specific warehouses can help in diversifying the product portfolio across warehouses. This insight can be valuable for optimizing inventory and distribution strategies to maximize revenue.

### Rolling Monthly Sales Growth

In [2]:
-- Calculate Rolling Monthly Sales Growth by Product Line
WITH MonthlySales AS (
    SELECT
        product_line,
        EXTRACT(MONTH FROM date) AS order_month,
        SUM(total) AS monthly_total_sales
    FROM Sales
    WHERE date BETWEEN '2021-06-01' AND '2021-08-31'
    GROUP BY product_line, order_month
),
MonthlyGrowth AS (
    SELECT
        product_line,
        order_month,
        monthly_total_sales,
        LAG(monthly_total_sales) OVER (PARTITION BY product_line ORDER BY order_month) AS prev_month_sales
    FROM MonthlySales
)
SELECT
    m.product_line,
    m.order_month,
    m.monthly_total_sales,
    CASE
        WHEN prev_month_sales IS NOT NULL THEN
            ROUND(((m.monthly_total_sales - prev_month_sales) / prev_month_sales) * 100, 2)
        ELSE
            0
    END AS monthly_sales_growth_percentage
FROM MonthlyGrowth m
ORDER BY m.product_line ASC, m.order_month ASC;


,product_line,order_month,monthly_total_sales,monthly_sales_growth_percentage
0,Breaking system,6,11469.92,0.00
1,Breaking system,7,15290.89,33.31
2,Breaking system,8,11589.34,-24.21
3,Electrical system,6,12010.70,0.00
4,Electrical system,7,14967.00,24.61
5,Electrical system,8,16635.01,11.14
6,Engine,6,11701.44,0.00
7,Engine,7,10344.96,-11.59
8,Engine,8,15898.98,53.69
9,Frame & body,6,24878.70,0.00


Few insights can be get from the table above, or the graph below:

1. **Breaking System**: This product line exhibited **significant fluctuations** in monthly sales growth during the specified date range. It started with a substantial growth rate of 33.31% from June to July but then experienced a sharp decline of -24.21% from July to August. **Understanding these fluctuations can help in adjusting inventory levels and marketing efforts accordingly.**

2. **Product Line Variability**: The monthly sales growth percentages differ significantly among product lines. For instance, while "Frame & body" experienced a decrease of -25.76% from June to July, it rebounded with a growth rate of 39.03% from July to August. In contrast, **"Suspension & traction"** saw relatively stable growth patterns with minor fluctuations, indicating a **steadier demand**.

,product_line,order_month,monthly_total_sales,monthly_sales_growth_percentage
0,Breaking system,6,11469.92,0.00
1,Breaking system,7,15290.89,33.31
2,Breaking system,8,11589.34,-24.21
3,Electrical system,6,12010.70,0.00
4,Electrical system,7,14967.00,24.61
5,Electrical system,8,16635.01,11.14
6,Engine,6,11701.44,0.00
7,Engine,7,10344.96,-11.59
8,Engine,8,15898.98,53.69
9,Frame & body,6,24878.70,0.00


### Cross-selling opportunities

This query would **pinpoint the most profitable product lines for different customer segments**, enabling data-driven decisions to optimize product offerings and marketing strategies tailored to each client type.

In [1]:
-- Query to find the product line with the highest total sales for each client type.
SELECT
    client_type,
    product_line,
    MAX(total_sales) AS highest_total_sales
FROM (
    SELECT
        client_type,
        product_line,
        SUM(total) AS total_sales
    FROM Sales
    GROUP BY client_type, product_line
) AS subquery
GROUP BY client_type, product_line;


,client_type,product_line,highest_total_sales
0,Retail,Breaking system,15219.25
1,Retail,Electrical system,21858.30
2,Retail,Engine,16709.33
3,Retail,Frame & body,29546.84
4,Retail,Miscellaneous,11418.01
5,Retail,Suspension & traction,34718.94
6,Wholesale,Breaking system,23130.90
7,Wholesale,Electrical system,21754.41
8,Wholesale,Engine,21236.05
9,Wholesale,Frame & body,39477.89



1. **Cross-Selling Opportunities**: The insights from this query reveal that **different product lines perform better for each client type**. This suggests the **potential for cross-selling** or targeted marketing strategies. For example, for retail clients who prefer "Suspension & traction," you might consider promoting related products or accessories to increase the average order value.

2. **Retail**: The product line **"Suspension & traction" stands out as the top-performing category** in terms of total sales, with a remarkable total sales value of $34,718.94. This suggests that retail clients have a strong preference for products in this category, and focusing on it can yield significant revenue for the business.

3. **Wholesale Clients**: For **wholesale clients, "Frame & body" emerges as the product line with the highest total sales**, totaling $39,477.89. This finding indicates that wholesale customers show a higher inclination toward products in this category. Understanding these client-type-specific preferences can guide inventory management and marketing efforts to cater to each segment more effectively.

### Average Order Value

In a retail business that offers a variety of product lines and accepts multiple payment methods, understanding customer behavior and optimizing revenue is crucial. The provided SQL query calculates the Average Order Value (AOV) for different combinations of product lines and payment methods specifically for retail clients. **AOV is a key metric that helps the business analyze the average amount spent by retail customers based on their product preferences and payment choices**.

By analyzing AOV by product line and payment method, the business can **gain insights into which product categories and payment methods are most popular among retail clients and which combinations result in higher average order values**. This information can inform marketing strategies, inventory management, and payment processing optimizations to maximize revenue and enhance the overall retail customer experience.

In [8]:
-- Query to calculate Average Order Value (AOV) by product line and payment method for retail clients
SELECT
    product_line,
    payment,
    AVG(total) AS average_order_value
FROM Sales
WHERE client_type = 'Retail'
GROUP BY product_line, payment
ORDER BY product_line, average_order_value DESC;


,product_line,payment,average_order_value
0,Breaking system,Cash,89.660800
1,Breaking system,Credit card,86.518200
2,Electrical system,Credit card,144.586718
3,Electrical system,Cash,121.560000
4,Engine,Cash,433.286667
5,Engine,Credit card,335.943095
6,Frame & body,Cash,253.856316
7,Frame & body,Credit card,226.821743
8,Miscellaneous,Credit card,124.588649
9,Miscellaneous,Cash,122.136111


### Warehouse Efficiency
Now, they want to now if the warehouse that supplies most orders is also the one that supplies the highest amount of sales 

In [9]:
-- Query to calculate which warehouse supplies the most orders
SELECT warehouse, COUNT(*) AS order_count
FROM Sales
GROUP BY warehouse
ORDER BY order_count DESC
LIMIT 1;


,warehouse,order_count
0,Central,480


In [10]:
-- Query to calculate which warehouse has the highest total sales
SELECT warehouse, SUM(total) AS total_sales
FROM Sales
GROUP BY warehouse
ORDER BY total_sales DESC
LIMIT 1;


,warehouse,total_sales
0,Central,141982.88


By prioritizing and optimizing the Cemtral Warehouse, we can not only meet customer demands promptly but also drive higher revenue, ultimately leading to a more successful and profitable operation. 

### Payment preferences
Understanding customer behavior and payment preferences is crucial for optimizing revenue and tailoring strategies to meet their needs effectively. This query aims to provide insights into our customer base by analyzing orders and total sales based on client type (retail or warehouse) and payment method. By doing so, we can gain a comprehensive understanding of how different customer segments engage with our products and services.

In [11]:
-- Query to analyze orders and total sales by client type and payment method
SELECT
    client_type,
    payment,
    COUNT(*) AS order_count,
    SUM(total) AS total_sales
FROM Sales
GROUP BY client_type, payment
ORDER BY client_type, total_sales DESC;


,client_type,payment,order_count,total_sales
0,Retail,Credit card,659,110271.57
1,Retail,Cash,116,19199.10
2,Wholesale,Transfer,225,159642.33


1. **Retail Customers**: It is evident that retail customers primarily prefer paying with credit cards, accounting for 659 orders with a total sales value of $110,271.57. Cash payments, while fewer in number (116 orders), contribute an additional $19,199.10 to our revenue.
2. **Wholesale Customers**: Wholesale clients, on the other hand, tend to favor transfer payments, with 225 orders and a substantial total sales value of $159,642.33. This data highlights the significance of accommodating different payment methods to cater to the diverse preferences of our clientele.